<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Data Preparation
Resave data as a set of tiff files in order to match Cell Tracking Challenge conventions which are expected by EmbedTrack.

In [7]:
import os
import sys

import numpy as np
import tensorflow as tf
from tifffile import imwrite

from deepcell.applications import NuclearSegmentation
from deepcell_tracking.trk_io import load_trks
from deepcell_tracking.isbi_utils import trk_to_isbi

sys.path.append('..')
import utils

In [2]:
source_data = '/notebooks/data/tracking/test.trks'
data_dir = '/notebooks/benchmarking/Baxter/data'
raw_dir = os.path.join(data_dir, 'raw')
seg_gt_dir = os.path.join(raw_dir, 'Analysis/Segmentation_GT')
seg_dc_dir = os.path.join(raw_dir, 'Analysis/Segmentation_DeepCell')
gt_dir = os.path.join(data_dir, 'GT')

for d in [raw_dir, seg_gt_dir, seg_dc_dir, gt_dir]:
    if not os.path.exists(d):
        os.makedirs(d)
        
model_url = 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearSegmentation-75.tar.gz'

Load the test split of the tracking data

In [3]:
data = load_trks(source_data)

Load the DeepCell nuclear segmentation model to test the algorithm on predicted instead of ground truth segmentations

In [4]:
archive_path = tf.keras.utils.get_file('NuclearSegmentation.tgz', model_url, extract=True, cache_subdir='models')
model_path = os.path.splitext(archive_path)[0]
model = tf.keras.models.load_model(model_path)

app = NuclearSegmentation(model)

2022-11-18 00:55:18.982931: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 00:55:19.682349: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1


Convert each batch of the test split to the standard ISBI format which is compatible with most of the models that we will test.

In [9]:
for batch_no in range(len(data['lineages'])):
    # Build subdirectories for data
    raw_subdir = os.path.join(raw_dir, '{:03}'.format(batch_no + 1))
    gt_subdir = os.path.join(gt_dir, '{:03}'.format(batch_no + 1))
    seg_gt_subdir = os.path.join(seg_gt_dir, '{:03}'.format(batch_no + 1))
    seg_dc_subdir = os.path.join(seg_dc_dir, '{:03}'.format(batch_no + 1))
    
    # Create directories if needed
    for d in (raw_subdir, gt_subdir, seg_gt_subdir, seg_dc_subdir):
        if not os.path.exists(d):
            os.makedirs(d)
                
    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]
    
    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)

    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]

    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames)]
    y = y[:len(good_frames)]
    
    # Need to translate lineages and adjust images to match restrictive ISBI format
    # Prepare output txt
    text_file = os.path.join(gt_subdir, 'man_track.txt')
    df = trk_to_isbi(lineage)
    df.to_csv(text_file, sep=' ', header=False, index=False)
    
    # Save each frame of the movie as an individual tif
    channel = 0 # These images should only have one channel
    for i in range(X.shape[0]):
        name_raw = os.path.join(raw_subdir, 't{:03}.tif'.format(i))
        name_seg_gt = os.path.join(seg_gt_subdir, 't{:03}.tif'.format(i))
        name_seg_dc = os.path.join(seg_dc_subdir, 't{:03}.tif'.format(i))
        name_gt = os.path.join(gt_subdir, 't{:03}.tif'.format(i))
        
        # Generate deepcell prediction
        pred = app.predict(X[i:i+1, ..., channel:channel+1])
        
        imwrite(name_raw, X[i, ..., channel])
        imwrite(name_seg_gt, y[i, ..., channel].astype('uint16'))
        imwrite(name_seg_dc, pred.astype('uint16'))
        imwrite(name_gt, y[i, ..., channel].astype('uint16'))

2022-11-18 00:58:07.723153: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima

# 2. Tracking

The steps for tracking will be performed in Matlab and the Baxter Algorithm GUI. You will need access to Matlab 2019b. The instructions below outline the general workflow for generating benchmarking results using the Baxter algorithm. However additional support for using the software is available in the user guide located in the KTH-SE folder (`KTH-SE/UserGuide/UserGuide.pdf`).

## Part 1 - Setup
1. Open the KTH-SE folder in Matlab
2. Open `BaxterAlgorithms.m` in the Matlab editor and then hit Run. This should open a GUI.
3. Within the GUI, select `Files > Open experiment > Browse...`. Navigate to the `data/raw` folder located within this Baxter folder.
4. Ensure that all files are selected in the list on the right hand side of the GUI.
## Part 2 - GT Segmentations
5. Select `Settings > Load Settings (Browse for files)`. Navigate to `Settings-SegmentationGT.csv` within the Baxter folder. In the following prompts, ensure that segmentations are applied to all images.
6. Select `Automated > Track`. In the pop-up window, change the `Save Version` to "GT" and press `Start`. 
7. Select `File > Export tracks to CTC format > RES tracks` and follow the prompts in order to export the tracking results to the standard CTC format. Make sure to select the "GT" version for export.
## Part 3 - DeepCell Segmentations
8. Select `Settings > Load Settings (Browse for files)`. Navigate to `Settings-SegmentationDeepCell.csv` within the Baxter folder. In the following prompts, ensure that segmentations are applied to all images.
6. Select `Automated > Track`. In the pop-up window, change the `Save Version` to "DeepCell" and press `Start`. 
7. Select `File > Export tracks to CTC format > RES tracks` and follow the prompts in order to export the tracking results to the standard CTC format. Make sure to select the "DeepCell" version for export.


# 3. Evaluation

In [47]:
import os
import re
import shutil
import subprocess

import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [49]:
data_dir = 'data'
gt_dir = os.path.join(data_dir, 'GT')
export_gt_dir = os.path.join(data_dir, 'raw/Analysis/CellDataGT/RES/')
export_dc_dir = os.path.join(data_dir, 'raw/Analysis/CellDataDeepCell/RES/')
ctc_gt = os.path.join(data_dir, 'SEG_GT')
ctc_dc = os.path.join(data_dir, 'SEG_PRED')

data_ids = os.listdir(gt_dir)

node_match_threshold = 0.6

ctc_software = '/notebooks/benchmarking/CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In order to utilize the CTC evaluation functions, the data must be reorganized to match CTC conventions.

In [32]:
# Copy the GT data over to appropriate folders
for target in [ctc_gt, ctc_dc]:
    for d in os.listdir(gt_dir):
        seg_dir = os.path.join(target, f'{d}_GT/SEG')
        tra_dir = os.path.join(target, f'{d}_GT/TRA')
        if not os.path.exists(seg_dir): os.makedirs(seg_dir) 
        if not os.path.exists(tra_dir): os.makedirs(tra_dir)
        
        for f in os.listdir(os.path.join(gt_dir, d)):
            start_path = os.path.join(gt_dir, d, f)
            
            # Only copy the txt file with tracks to TRA dir
            if f.endswith('txt'):
                end_path = os.path.join(tra_dir, f)
                shutil.copy(start_path, end_path)
            else:
                # Copy to seg_dir and modify name
                end_path = os.path.join(seg_dir, f'man_seg{f[1:]}')
                shutil.copy(start_path, end_path)

                # Copy to tra dir and modify name
                end_path = os.path.join(tra_dir, f'man_track{f[1:]}')
                shutil.copy(start_path, end_path)
                
# Copy over results
for target, start_dirs in zip([ctc_gt, ctc_dc], [export_gt_dir, export_dc_dir]):
    for d in os.listdir(start_dirs):
        start_dir = os.path.join(start_dirs, d)
        end_dir = os.path.join(target, d)
        shutil.copytree(start_dir, end_dir)

In [50]:
benchmarks = []

for results_dir, s in zip([ctc_gt, ctc_dc], ['GT', 'Deepcell']):
    for data_id in data_ids:
        print(data_id)
        results = {
            'model': f'Baxter - {s}',
            'data_id': data_id
        }
        gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')

        # Deepcell benchmarking
        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
        results.update(m.stats)
        
        # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout
            
            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, results_dir, data_id)
                print(outstring.decode('utf-8'))
        
        benchmarks.append(results)
        
df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

011
missed node 21_42 division completely
missed node 38_5 division completely
missed node 43_4 division completely
missed node 50_42 division completely
missed node 73_5 division completely
002
009
missed node 5_6 division completely
missed node 10_66 division completely
missed node 15_66 division completely
missed node 25_57 division completely
30_37 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 55_27 division completely
missed node 76_44 division completely
missed node 94_36 division completely
missed node 104_34 division completely
missed node 112_6 division completely
missed node 118_0 division completely
missed node 121_42 division completely
missed node 125_22 division completely
missed node 127_51 division completely
missed node 128_43 division completely
missed node 132_23 division completely
missed node 140_10 division completely
missed node 144_28 division completely
missed node 147_33 division completely
missed node 160_64 division completely
miss

In [51]:
df

,model,data_id,correct_division,mismatch_division,false_positive_division,false_negative_division,total_divisions,aa_tp,aa_total,te_tp,te_total,DET,SEG,TRA
0,Baxter - GT,011,10,0,0,5,15,3831,3833,3966,3975,0.997811,0.996626,0.997014
1,Baxter - GT,002,1,0,0,0,1,1075,1075,1108,1109,0.999098,0.999098,0.999213
2,Baxter - GT,009,33,1,9,23,57,14656,14806,15013,15174,0.997832,0.995769,0.996396
3,Baxter - GT,003,5,0,2,3,8,2043,2055,2108,2122,0.997408,0.997169,0.997143
4,Baxter - GT,012,6,2,0,8,16,9788,9796,10064,10097,0.991186,0.989493,0.989699
5,Baxter - GT,007,0,0,0,1,1,199,199,204,206,0.997573,0.995124,0.996612
6,Baxter - GT,001,2,0,0,1,3,997,997,1030,1030,1.000000,1.000000,0.999915
7,Baxter - GT,006,0,0,0,0,0,383,383,396,397,0.994962,0.994962,0.995599
8,Baxter - GT,010,31,3,2,20,54,9045,9090,9302,9357,0.998130,0.997559,0.997173
9,Baxter - GT,004,1,0,0,0,1,521,521,538,540,0.996296,0.996296,0.996766
